# <center>手写数字分类识别入门体验教程</center>

## 实现一个图片分类应用
## 概述
下面我们通过一个实际样例，带领大家体验MindSpore基础的功能，对于一般的用户而言，完成整个样例实践会持续20~30分钟。

本例子会实现一个简单的图片分类的功能，整体流程如下：

1、处理需要的数据集，这里使用了MNIST数据集。

2、定义一个网络，这里我们使用LeNet网络。

3、定义损失函数和优化器。

4、加载数据集并进行训练，训练完成后，查看结果及保存模型文件。

5、加载保存的模型，进行推理。

6、验证模型，加载测试数据集和训练后的模型，验证结果精度。

说明：<br/>你可以在这里找到完整可运行的样例代码：https://gitee.com/mindspore/docs/blob/r0.5/tutorials/tutorial_code/lenet.py

## 一、训练的数据集下载

#### 方法一：
从以下网址下载，并将数据包解压缩后放至Jupyter的工作目录下：<br/>训练数据集：{"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
<br/>测试数据集：{"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}<br/>我们用下面代码查询jupyter的工作目录。

In [ ]:
import os
os.getcwd()

训练数据集放在----Jupyter工作目录+\MNIST_Data\train\，此时train文件夹内应该包含两个文件，train-images-idx3-ubyte和train-labels-idx1-ubyte <br/>测试数据集放在----Jupyter工作目录+\MNIST_Data\test\，此时test文件夹内应该包含两个文件，t10k-images-idx3-ubyte和t10k-labels-idx1-ubyte

#### 方法二：
直接执行下面代码，会自动进行训练集的下载与解压，但是整个过程根据网络好坏情况会需要花费几分钟时间。

In [ ]:
# Network request module, data download module, decompression module
import urllib.request   
from urllib.parse import urlparse
import gzip 

def unzipfile(gzip_path):
    """unzip dataset file
    Args:
        gzip_path: dataset file path
    """
    open_file = open(gzip_path.replace('.gz',''), 'wb')
    gz_file = gzip.GzipFile(gzip_path)
    open_file.write(gz_file.read())
    gz_file.close()
    
def download_dataset():
    """Download the dataset from http://yann.lecun.com/exdb/mnist/."""
    print("******Downloading the MNIST dataset******")
    train_path = "./MNIST_Data/train/" 
    test_path = "./MNIST_Data/test/"
    train_path_check = os.path.exists(train_path)
    test_path_check = os.path.exists(test_path)
    if train_path_check == False and test_path_check == False:
        os.makedirs(train_path)
        os.makedirs(test_path)
    train_url = {"http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"}
    test_url = {"http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz", "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"}
    
    for url in train_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(train_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)
            
    for url in test_url:
        url_parse = urlparse(url)
        # split the file name from url
        file_name = os.path.join(test_path,url_parse.path.split('/')[-1])
        if not os.path.exists(file_name.replace('.gz', '')):
            file = urllib.request.urlretrieve(url, file_name)
            unzipfile(file_name)
            os.remove(file_name)

download_dataset()

这样就完成了数据集的下载解压缩工作。

## 二、处理MNIST数据集

由于我们后面会采用LeNet这样的卷积神经网络对数据集进行训练，而采用LeNet在训练数据时，对数据格式是有所要求的，所以接下来的工作需要我们先查看数据集内的数据是什么样的，这样才能构造一个针对性的数据转换函数，将数据集数据转换成符合训练要求的数据形式。

更多的LeNet网络的介绍不在此赘述，希望详细了解LeNet网络，可以查询http://yann.lecun.com/exdb/lenet/ 。

### 查看原始数据集数据

In [ ]:
from mindspore import context
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import mindspore.dataset as ds

context.set_context(mode=context.GRAPH_MODE, device_target="CPU") # Windows version, set to use CPU for graph calculation
train_data_path = "./MNIST_Data/train"
test_data_path = "./MNIST_Data/test"
mnist_ds = ds.MnistDataset(train_data_path) # Load training dataset
print('The type of mnist_ds:', type(mnist_ds))
print("Number of pictures contained in the mnist_ds：",mnist_ds.get_dataset_size()) # 60000 pictures in total

dic_ds = mnist_ds.create_dict_iterator() # Convert dataset to dictionary type
item = dic_ds.get_next()
img = item["image"]
label = item["label"]

print("The item of mnist_ds:", item.keys()) # Take a single data to view the data structure, including two keys, image and label
print("Tensor of image in item:", img.shape) # View the tensor of image (28,28,1)
print("The label of item:", label)

plt.imshow(np.squeeze(img))
plt.title("number:%s"% item["label"])
plt.show()

从上面的运行情况我们可以看到,训练数据集train-images-idx3-ubyte和train-labels-idx1-ubyte对应的是6万张图片和6万个数字下标，载入数据后经过create_dict_iterator()转换字典型的数据集，取其中的一个数据查看，这是一个key为image和label的字典，其中的image的张量(高度28，宽度28，通道1)和label为对应图片的数字。

### 数据处理

数据集对于训练非常重要，好的数据集可以有效提高训练精度和效率。在加载数据集前，我们通常会对数据集进行一些处理。
#### 定义数据集及数据操作
我们定义一个函数create_dataset()来创建数据集。在这个函数中，我们定义好需要进行的数据增强和处理操作：
<br/>1、定义数据集。
<br/>2、定义进行数据增强和处理所需要的一些参数。
<br/>3、根据参数，生成对应的数据增强操作。
<br/>4、使用map()映射函数，将数据操作应用到数据集。
<br/>5、对生成的数据集进行处理。

In [ ]:
# Data processing module
import mindspore.dataset.transforms.vision.c_transforms as CV
import mindspore.dataset.transforms.c_transforms as C
from mindspore.dataset.transforms.vision import Inter
from mindspore.common import dtype as mstype


def create_dataset(data_path, batch_size=32, repeat_size=1,
                   num_parallel_workers=1):
    """ create dataset for train or test
    Args:
        data_path: Data path
        batch_size: The number of data records in each group
        repeat_size: The number of replicated data records
        num_parallel_workers: The number of parallel workers
    """
    # define dataset
    mnist_ds = ds.MnistDataset(data_path)

    # Define some parameters needed for data enhancement and rough justification
    resize_height, resize_width = 32, 32
    rescale = 1.0 / 255.0
    shift = 0.0
    rescale_nml = 1 / 0.3081
    shift_nml = -1 * 0.1307 / 0.3081

    # According to the parameters, generate the corresponding data enhancement method
    resize_op = CV.Resize((resize_height, resize_width), interpolation=Inter.LINEAR)  # Resize images to (32, 32) by bilinear interpolation
    rescale_nml_op = CV.Rescale(rescale_nml, shift_nml) # normalize images
    rescale_op = CV.Rescale(rescale, shift) # rescale images
    hwc2chw_op = CV.HWC2CHW() # change shape from (height, width, channel) to (channel, height, width) to fit network.
    type_cast_op = C.TypeCast(mstype.int32) # change data type of label to int32 to fit network

    # Using map () to apply operations to a dataset
    mnist_ds = mnist_ds.map(input_columns="label", operations=type_cast_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=resize_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=rescale_nml_op, num_parallel_workers=num_parallel_workers)
    mnist_ds = mnist_ds.map(input_columns="image", operations=hwc2chw_op, num_parallel_workers=num_parallel_workers)
    # Process the generated dataset
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)  # 10000 as in LeNet train script
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds


其中<br/>
batch_size：每组包含的数据个数，现设置每组包含32个数据。
<br/>repeat_size：数据集复制的数量。
<br/>先进行shuffle、batch操作，再进行repeat操作，这样能保证1个epoch内数据不重复。

接下来我们查看将要进行训练的数据集内容是什么样的。

首先，查看数据集内包含多少组数据。

In [ ]:
datas = create_dataset(train_data_path) # Process the train dataset
print('Number of groups in the dataset:', datas.get_dataset_size()) # Number of query dataset groups

其次，取出其中一组数据，查看包含的key，图片数据的张量，以及下标labels的值。

In [ ]:
data = datas.create_dict_iterator().get_next() # Take a set of datasets
print(data.keys())
images = data["image"] # Take out the image data in this dataset
labels = data["label"] # Take out the label (subscript) of this data set
print('Tensor of image:', images.shape) # Query the tensor of images in each dataset (32,1,32,32)
print('labels:', labels)

最后，查看image的图像和下标对应的值。

In [ ]:
count = 1
for i in images:
    plt.subplot(4, 8, count) 
    plt.imshow(np.squeeze(i))
    plt.title('num:%s'%labels[count-1])
    plt.xticks([])
    count += 1
    plt.axis("off")
plt.show() # Print a total of 32 pictures in the group

通过上述三个查询操作，看到经过变换后的图片，数据集内分成了1875组数据，每组数据中含有32张图片，每张图片像数值为32×32，数据全部准备好后，就可以进行下一步的数据训练了。

## 三、构造神经网络

在对手写字体识别上，通常采用卷积神经网络架构（CNN）进行学习预测，最经典的属1998年由Yann LeCun创建的LeNet5架构，<br/>其中分为：<br/>1、输入层；<br/>2、卷积层C1；<br/>3、池化层S2；<br/>4、卷积层C3；<br/>5、池化层S4；<br/>6、全连接F6；<br/>7、全连接；<br/>8、全连接OUTPUT。<br/>结构示意如下图:

### LeNet5结构图

<img src="https://img-blog.csdnimg.cn/20190305161316701.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L21tbV9qc3c=,size_16,color_FFFFFF,t_70" alt="LeNet5">

在构建LeNet5前，我们需要对全连接层以及卷积层进行初始化。

TruncatedNormal：参数初始化方法，MindSpore支持TruncatedNormal、Normal、Uniform等多种参数初始化方法，具体可以参考MindSpore API的mindspore.common.initializer模块说明。

初始化示例代码如下：

In [ ]:
import mindspore.nn as nn
from mindspore.common.initializer import TruncatedNormal

# Initialize 2D convolution function
def conv(in_channels, out_channels, kernel_size, stride=1, padding=0):
    """Conv layer weight initial."""
    weight = weight_variable()
    return nn.Conv2d(in_channels, out_channels,
                     kernel_size=kernel_size, stride=stride, padding=padding,
                     weight_init=weight, has_bias=False, pad_mode="valid")

# Initialize full connection layer
def fc_with_initialize(input_channels, out_channels):
    """Fc layer weight initial."""
    weight = weight_variable()
    bias = weight_variable()
    return nn.Dense(input_channels, out_channels, weight, bias)

# Set truncated normal distribution
def weight_variable():
    """Weight initial."""
    return TruncatedNormal(0.02)

使用MindSpore定义神经网络需要继承mindspore.nn.cell.Cell，Cell是所有神经网络（Conv2d等）的基类。

神经网络的各层需要预先在\_\_init\_\_()方法中定义，然后通过定义construct()方法来完成神经网络的前向构造，按照LeNet5的网络结构，定义网络各层如下：

In [ ]:
class LeNet5(nn.Cell):
    """Lenet network structure."""
    # define the operator required
    def __init__(self):
        super(LeNet5, self).__init__()
        self.batch_size = 32 # 32 pictures in each group
        self.conv1 = conv(1, 6, 5) # Convolution layer 1, 1 channel input (1 Figure), 6 channel output (6 figures), convolution core 5 * 5
        self.conv2 = conv(6, 16, 5) # Convolution layer 2,6-channel input, 16 channel output, convolution kernel 5 * 5
        self.fc1 = fc_with_initialize(16 * 5 * 5, 120)
        self.fc2 = fc_with_initialize(120, 84)
        self.fc3 = fc_with_initialize(84, 10)
        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

    # use the preceding operators to construct networks
    def construct(self, x):
        x = self.conv1(x) # 1*32*32-->6*28*28
        x = self.relu(x) # 6*28*28-->6*14*14
        x = self.max_pool2d(x) # Pool layer
        x = self.conv2(x) # Convolution layer
        x = self.relu(x) # Function excitation layer
        x = self.max_pool2d(x) # Pool layer
        x = self.flatten(x) # Dimensionality reduction
        x = self.fc1(x) # Full connection
        x = self.relu(x) # Function excitation layer
        x = self.fc2(x) # Full connection
        x = self.relu(x) # Function excitation layer
        x = self.fc3(x) # Full connection
        return x

构建完成后，我们将LeNet5的整体参数打印出来查看一下。

In [ ]:
network = LeNet5()
print(network)

In [ ]:
param = network.trainable_params()
param

## 四、搭建训练网络并进行训练

构建完成神经网络后，就可以着手进行训练网络的构建，模型训练函数为Model.train(),参数主要包含:
<br/>1、圈数epoch size（每圈需要遍历完成1875组图片）;
<br/>2、数据集ds_train;
<br/>3、回调函数callbacks包含ModelCheckpoint、LossMonitor、SummaryStepckpoint_cb,Callback模型检测参数;
<br/>4、底层数据通道dataset_sink_mode，此参数默认True需设置成False，因为此功能不支持CPU模式。

In [ ]:
# Training and testing related modules
import argparse
from mindspore import Tensor
from mindspore.train.serialization import load_checkpoint, load_param_into_net
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig, LossMonitor,SummaryStep,Callback
from mindspore.train import Model
from mindspore.nn.metrics import Accuracy
from mindspore.nn.loss import SoftmaxCrossEntropyWithLogits

def train_net(model, epoch_size, mnist_path, repeat_size, ckpoint_cb, step_loss_info):
    """Define the training method."""
    print("============== Starting Training ==============")
    # load training dataset
    ds_train = create_dataset(os.path.join(mnist_path, "train"), 32, repeat_size)
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, LossMonitor(), step_loss_info], dataset_sink_mode=False)

自定义一个存储每一步训练的step和对应loss值的类Step_loss_info()，并继承了Callback类，可以自定义训练过程中的处理措施，非常方便，等训练完成后，可将数据绘图查看loss的变化情况。

In [ ]:
# Custom callback function
class Step_loss_info(Callback):
    def step_end(self, run_context):
        cb_params = run_context.original_args()
        # step_ Loss dictionary for saving loss value and step number information
        step_loss["loss_value"].append(str(cb_params.net_outputs))
        step_loss["step"].append(str(cb_params.cur_step_num))

### 定义损失函数及优化器
基本概念
在进行定义之前，先简单介绍损失函数及优化器的概念。
<br/>损失函数：又叫目标函数，用于衡量预测值与实际值差异的程度。深度学习通过不停地迭代来缩小损失函数的值。定义一个好的损失函数，可以有效提高模型的性能。
<br/>优化器：用于最小化损失函数，从而在训练过程中改进模型。
<br/>定义了损失函数后，可以得到损失函数关于权重的梯度。梯度用于指示优化器优化权重的方向，以提高模型性能。
<br/>定义损失函数。
<br/>MindSpore支持的损失函数有SoftmaxCrossEntropyWithLogits、L1Loss、MSELoss等。这里使用SoftmaxCrossEntropyWithLogits损失函数。

In [ ]:
import os

if os.name == "nt":
    os.system('del/f/s/q *.ckpt *.meta')# Clean up old run files before in Windows
else:
    os.system('rm -f *.ckpt *.meta *.pb')# Clean up old run files before in Linux

lr = 0.01 # learning rate
momentum = 0.9 #

# create the network
network = LeNet5()

# define the optimizer
net_opt = nn.Momentum(network.trainable_params(), lr, momentum)


# define the loss function
net_loss = SoftmaxCrossEntropyWithLogits(is_grad=False, sparse=True, reduction='mean')
# define the model
model = Model(network, net_loss, net_opt, metrics={"Accuracy": Accuracy()} )


epoch_size = 1
mnist_path = "./MNIST_Data"

config_ck = CheckpointConfig(save_checkpoint_steps=125, keep_checkpoint_max=16)
# save the network model and parameters for subsequence fine-tuning

ckpoint_cb = ModelCheckpoint(prefix="checkpoint_lenet", config=config_ck)
# group layers into an object with training and evaluation features
step_loss = {"step": [], "loss_value": []}
# step_ Loss dictionary for saving loss value and step number information
step_loss_info = Step_loss_info()
# save the steps and loss value
repeat_size = 1
train_net(model, epoch_size, mnist_path, repeat_size, ckpoint_cb, step_loss_info)


训练完成后，能在Jupyter的工作路径上生成多个模型文件，名称具体含义checkpoint_{网络名称}-{第几个epoch}_{第几个step}.ckpt。

#### 查看损失函数随着训练步数的变化情况

In [ ]:
steps = step_loss["step"]
loss_value = step_loss["loss_value"]
steps = list(map(int, steps))
loss_value = list(map(float, loss_value))
plt.plot(steps, loss_value, color="red")
plt.xlabel("Steps")
plt.ylabel("Loss_value")
plt.title("Loss function value change chart")
plt.show()

从上面可以看出来大致分为三个阶段：

阶段一：开始训练loss值在2.2上下浮动，训练收益感觉并不明显。

阶段二：训练到某一时刻，loss值减少迅速，训练收益大幅增加。

阶段三：loss值收敛到一定小的值后，loss值开始振荡在一个小的区间上无法趋0，再继续增加训练并无明显收益，至此训练结束。

##  五、数据测试验证模型精度

搭建测试网络的过程主要为：<br/>1、载入模型.cptk文件中的参数param；<br/>2、将参数param载入到神经网络LeNet5中；<br/>3、载入测试数据集；<br/>4、调用函数model.eval()传入参数测试数据集ds_eval，就生成模型checkpoint_lenet-1_1875.ckpt的精度值。<br/>dataset_sink_mode表示数据集下沉模式，不支持CPU，所以这里设置成False。

In [ ]:
def test_net(network, model, mnist_path):
    """Define the evaluation method."""
    print("============== Starting Testing ==============")
    # load the saved model for evaluation
    param_dict = load_checkpoint("checkpoint_lenet-1_1875.ckpt")
    # load parameter to the network
    load_param_into_net(network, param_dict)
    # load testing dataset
    ds_eval = create_dataset(os.path.join(mnist_path, "test"))
    acc = model.eval(ds_eval, dataset_sink_mode=False)
    print("============== Accuracy:{} ==============".format(acc))

test_net(network, model, mnist_path)

经过1875步训练后生成的模型精度超过95%，模型优良。
我们可以看一下模型随着训练步数变化，精度随之变化的情况。

acc_model_info()函数是将每125步的保存的模型，调用model.eval()函数将测试出的精度返回到步数列表和精度列表，如下：

In [ ]:
def acc_model_info(network, model, mnist_path, model_numbers):
    """Define the plot info method"""
    step_list = []
    acc_list = []
    for i in range(1, model_numbers+1):
        # load the saved model for evaluation
        param_dict = load_checkpoint("checkpoint_lenet-1_{}.ckpt".format(str(i*125)))
        # load parameter to the network
        load_param_into_net(network, param_dict)
        # load testing dataset
        ds_eval = create_dataset(os.path.join(mnist_path, "test"))
        acc = model.eval(ds_eval, dataset_sink_mode=False)
        acc_list.append(acc['Accuracy'])
        step_list.append(i*125)
    return step_list,acc_list

# Draw line chart according to training steps and model accuracy
l1,l2 = acc_model_info(network, model, mnist_path, 15)
plt.xlabel("Model of Steps")
plt.ylabel("Model accuracy")
plt.title("Model accuracy variation chart")
plt.plot(l1, l2, 'red')
plt.show()

从图中可以看出训练得到的模型精度变化分为三个阶段：1、缓慢上升，2、迅速上升，3、缓慢上升趋近于不到1的某个值时附近振荡，说明随着训练数据的增加，会对模型精度有着正相关的影响，但是随着精度到达一定程度，训练收益会降低。

### 六、模型预测应用

我们尝试使用生成的模型应用到分类预测单个或者单组图片数据上，具体步骤如下：

1、需要将要测试的数据转换成适应LeNet5的数据类型。
<br/>2、提取出image的数据。
<br/>3、使用函数model.predict()预测image对应的数字。需要说明的是predict返回的是image对应0-9的概率值。
<br/>4、调用plot_pie()将预测的各数字的概率显示出来。负概率的数字会被去掉。

载入要测试的数据集并调用create_dataset()转换成符合格式要求的数据集，并选取其中一组32张图片进行预测。

In [ ]:
ds_test = create_dataset(test_data_path).create_dict_iterator()
data = ds_test.get_next()
images = data["image"]
labels = data["label"] # The subscript of data picture is the standard for us to judge whether it is correct or not

output =model.predict(Tensor(data['image']))
# The predict function returns the probability of 0-9 numbers corresponding to each picture
prb = output.asnumpy()
pred = np.argmax(output.asnumpy(), axis=1)
err_num = []
index = 1
for i in range(len(labels)):
    plt.subplot(4, 8, i+1)
    color = 'blue' if pred[i] == labels[i] else 'red'
    plt.title("pre:{}".format(pred[i]), color=color)
    plt.imshow(np.squeeze(images[i]))
    plt.axis("off")
    if color == 'red':
        index = 0
        # Print out the wrong data identified by the current group
        print("Row {}, column {} is incorrectly identified as {}, the correct value should be {}".format(int(i/8)+1, i%8+1, pred[i], labels[i]), '\n')
if index:
    print("All the figures in this group are predicted correctly！")
print(pred, "<--Predicted figures") # Print the numbers recognized by each group of pictures
print(labels, "<--The right number") # Print the subscript corresponding to each group of pictures
plt.show()

构建一个概率分析的饼图函数。

备注：prb为上一段代码中，存储这组数对应的数字概率。

In [ ]:
# define the pie drawing function of probability analysis
def plot_pie(prbs):
    dict1 = {}
    # Remove the negative number and build the dictionary dict1. The key is the number and the value is the probability value
    for i in range(10):
        if prbs[i] > 0:
            dict1[str(i)] = prbs[i]

    label_list = dict1.keys()    # Label of each part
    size = dict1.values()    # Size of each part
    colors = ["red", "green", "pink", "blue", "purple", "orange", "gray"] # Building a round cake pigment Library
    color = colors[: len(size)]# Color of each part
    plt.pie(size, colors=color, labels=label_list, labeldistance=1.1, autopct="%1.1f%%", shadow=False, startangle=90, pctdistance=0.6)
    plt.axis("equal")    # Set the scale size of x-axis and y-axis to be equal
    plt.legend()
    plt.title("Image classification")
    plt.show()
    
    
for i in range(2):
    print("Figure {} probability of corresponding numbers [0-9]:\n".format(i+1), prb[i])
    plot_pie(prb[i])

以上过程就是这次手写数字分类训练的全部体验过程。